In [1]:
# dessa vez a gente vai usar as definições do sage,
# em particular a implementação de grafos.

from sage.all import *
from docplex.mp.model import Model
import itertools as it

In [2]:
# vamos criar uma função que, dado um grafo,
# retorna um modelo matemático para o problema do emparelhamento máximo

def maximum_matching(G):
    # criamos o modelo
    mdl = Model()
    # criamos as variáveis
    # as arestas têm uma terceira componente, que pode ser usada como peso,
    # e como não precisamos de peso, vamos ignorá-la
    x = {(u, v) : mdl.binary_var() for (u, v, _) in G.edges()}
    # uma restrição para cada vértice
    for v in G.vertices():
        mdl.add_constraint(
            sum(x[a, b] for (a, b, _) in G.edges_incident(v)) <= 1)
    # por fim, a função-objetivo de maximização
    mdl.maximize(sum(x[u, v] for (u, v, _) in G.edges()))
    # retornamos o modelo pronto, junto com suas variáveis
    return mdl, x

In [3]:
# agora podemos criar um grafo e testar nosso modelo
# isto cria um grafo aleatório com 100 vértices,
# e chance 0.5 de ocorrência de uma aresta entre cada par de vértices
G = graphs.random.RandomGNP(100, 0.5)
mdl, x = maximum_matching(G)

In [4]:
# resolvemos o modelo
sol = mdl.solve(log_output=True)
sol.display()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
Found incumbent of value 0.000000 after 0.00 sec. (0.06 ticks)
Tried aggregator 1 time.
Reduced MIP has 100 rows, 2497 columns, and 4994 nonzeros.
Reduced MIP has 2497 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (5.76 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 100 rows, 2497 columns, and 4994 nonzeros.
Reduced MIP has 2497 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (6.32 ticks)
Probing time = 0.01 sec. (3.43 ticks)
Clique table members: 100.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.02 sec. (7.56 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer

In [5]:
# agora vamos tomar o conjunto de arestas escolhidas
# pelo modelo como um emparelhamento máximo
# vamos representar as arestas por conjuntos
# para podermos checar que elas são disjuntas

matching = [set([u, v]) for (u, v, _) in G.edges() if x[u, v].solution_value == 1]
len(matching)

50

In [6]:
# vamos checar que as arestas selecionadas de fato formam um emparelhamento

def is_matching(edges):
    return all(e1.isdisjoint(e2)
               for (e1, e2) in it.combinations(edges, 2))

In [7]:
# e agora constatamos que as arestas selecionadas de fato formam um emparelhamento

is_matching(matching)

True

In [8]:
# agora sabemos que de fato o modelo selecionou um emparelhamento,
# mas terá sido um emparelhamento máximo?

def is_maximum_matching(G, edges):
    G_edges_as_sets = [set([u, v]) for (u, v, _) in G.edges()]
    cannot_expand = lambda e1: not all(e1.isdisjoint(e2) for e2 in edges)
    cannot_be_expanded = all(cannot_expand(e) for e in G_edges_as_sets)
    return is_matching(edges) and cannot_be_expanded

In [9]:
# e verificamos que de fato temos um emparelhamento máximo
is_maximum_matching(G, matching)

True